<a href="https://colab.research.google.com/github/sergiocostaifes/PPCOMP_DM/blob/main/03_window_5min_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03 — Construção de Janelas Temporais de 5 Minutos

## 1. Objetivo

Este notebook implementa a discretização temporal do dataset em janelas fixas de 5 minutos (300 segundos), criando a base estrutural para modelagem de séries temporais e detecção de episódios de falha.

A partir do timestamp relativo `t_rel_us`, define-se:

**bucket_id = floor(t_rel_us / 300000000)**

onde 300000000 representa 5 minutos em microssegundos.

---

## 2. Estrutura Metodológica

### 2.1 Leitura do Dataset Limpo

- Arquivo de entrada: `google_trace_clean.parquet`
- Total esperado de registros: 349151
- Colunas mínimas requeridas:
  - `t_rel_us`
  - `machine_id`
  - `collection_id`
  - `event`
  - `failed`

Esta etapa garante integridade estrutural antes da agregação temporal.

---

### 2.2 Construção do Minute Bucket

Para cada registro:

- Cálculo de `bucket_id`
- Cálculo de `bucket_start_us`
- Verificação do intervalo mínimo e máximo de buckets
- Contagem de buckets distintos

Esta etapa transforma eventos individuais em unidades temporais discretas.

---

### 2.3 Agregações por Janela (window_5min_base)

Para cada bucket são calculadas as seguintes métricas:

#### Métricas de Volume
- `n_events`
- `n_failed`
- `n_machines`
- `n_collections`

#### Métricas de Intensidade
- `mean_priority`
- `mean_req_cpus`
- `mean_req_mem`

#### Métricas de Tipologia de Evento
- `event_FAIL_count`
- `event_SCHEDULE_count`
- `event_FINISH_count`
- `event_ENABLE_count`
- `event_LOST_count`
- `event_EVICT_count`
- `event_KILL_count`

---

### 2.4 Construção da Série Temporal Contínua

- Reindexação do intervalo completo de buckets
- Buckets sem eventos recebem zero nas métricas de contagem
- Métricas contínuas permanecem como NaN
- Estrutura preparada para operações de rolling e modelagem supervisionada

---

## 3. Artefatos Gerados

- `window_5min_base.parquet`
- `window_5min_series.parquet`
- `03_window_5min_base_summary.json`

---

## 4. Resultados Esperados

- Aproximadamente 8.9 mil janelas temporais
- Série contínua com exatamente `(max_bucket - min_bucket + 1)` linhas
- Estrutura pronta para:
  - Detecção de episódios (Notebook 04)
  - Feature Engineering (Notebook 05)
  - Modelagem supervisionada (Notebook 07)

---

## 5. Interpretação Metodológica

A discretização temporal:

- Preserva a dinâmica de falhas ao longo do tempo
- Permite análise de densidade de eventos
- Torna o problema tratável como série temporal supervisionada
- Separa claramente a estrutura temporal da engenharia de atributos

Este notebook representa a transição do nível de eventos individuais para o nível de comportamento agregado do sistema.

In [4]:
# ============================================================
# 03_window_5min_base.ipynb
# ============================================================

# =========================
# Bootstrap Seguro
# =========================
from pathlib import Path
import sys, subprocess, importlib

# Mount seguro (não quebra se já estiver montado)
if not Path("/content/drive/MyDrive").exists():
    from google.colab import drive
    drive.mount("/content/drive")
else:
    print("[Bootstrap] Drive já montado.")

REPO_DIR = Path("/content/drive/MyDrive/Mestrado/PPCOMP_DM")
GITHUB_REPO = "https://github.com/sergiocostaifes/PPCOMP_DM.git"

if not REPO_DIR.exists():
    REPO_DIR.parent.mkdir(parents=True, exist_ok=True)
    subprocess.run(["git", "clone", GITHUB_REPO, str(REPO_DIR)], check=True)

repo_str = str(REPO_DIR)
if repo_str not in sys.path:
    sys.path.insert(0, repo_str)

importlib.invalidate_caches()

from src.paths import PROCESSED_PATH, FEATURES_PATH, REPORTS_PATH, ensure_dirs
ensure_dirs()

def log(msg):
    print(f"[03_window_5min_base] {msg}")


# =========================
# Parâmetros
# =========================
BUCKET_SEC = 5 * 60
BUCKET_US = BUCKET_SEC * 1_000_000

TOP_EVENTS = ["FAIL", "SCHEDULE", "FINISH", "ENABLE", "LOST", "EVICT", "KILL"]


# =========================
# 1) Leitura
# =========================
import pandas as pd
import numpy as np
import json

CLEAN_PARQUET = PROCESSED_PATH / "google_trace_clean.parquet"
assert CLEAN_PARQUET.exists(), f"Arquivo não encontrado: {CLEAN_PARQUET}"

df = pd.read_parquet(CLEAN_PARQUET)

log(f"Shape entrada: {df.shape}")

required_cols = ["t_rel_us", "machine_id", "collection_id", "event", "failed"]
missing = [c for c in required_cols if c not in df.columns]
assert not missing, f"Colunas ausentes: {missing}"

df["t_rel_us"] = pd.to_numeric(df["t_rel_us"], errors="coerce")
df = df.dropna(subset=["t_rel_us"]).copy()
df["t_rel_us"] = df["t_rel_us"].astype("int64")


# =========================
# 2) Minute Bucket
# =========================
df["bucket_id"] = (df["t_rel_us"] // BUCKET_US).astype("int64")
df["bucket_start_us"] = df["bucket_id"] * BUCKET_US

log(f"bucket range: {df['bucket_id'].min()}..{df['bucket_id'].max()}")
log(f"Buckets distintos: {df['bucket_id'].nunique()}")


# =========================
# 3) resource_request (leve)
# =========================
import ast

def parse_dict(x):
    if isinstance(x, dict):
        return x
    if isinstance(x, str):
        s = x.strip()
        if not s:
            return None
        # 1) JSON padrão
        try:
            return json.loads(s)
        except Exception:
            pass
        # 2) Fallback p/ strings tipo "{'cpus': 1, 'memory': 2}"
        try:
            v = ast.literal_eval(s)
            return v if isinstance(v, dict) else None
        except Exception:
            return None
    return None

if "resource_request" in df.columns:
    rr = df["resource_request"].map(parse_dict)
    df["req_cpus"] = rr.map(lambda d: d.get("cpus") if isinstance(d, dict) else np.nan)
    df["req_mem"]  = rr.map(lambda d: d.get("memory") if isinstance(d, dict) else np.nan)
else:
    df["req_cpus"] = np.nan
    df["req_mem"] = np.nan


# =========================
# 4) Agregações
# =========================
base = (
    df.groupby("bucket_id", as_index=False)
      .agg(
          bucket_start_us=("bucket_start_us", "min"),
          n_events=("event", "size"),
          n_failed=("failed", "sum"),
          n_machines=("machine_id", "nunique"),
          n_collections=("collection_id", "nunique"),
          mean_priority=("priority", "mean"),
          mean_req_cpus=("req_cpus", "mean"),
          mean_req_mem=("req_mem", "mean"),
      )
)

evt_counts = (
    df[df["event"].isin(TOP_EVENTS)]
      .groupby(["bucket_id", "event"], observed=False)
      .size()
      .unstack(fill_value=0)
)

for ev in TOP_EVENTS:
    if ev not in evt_counts.columns:
        evt_counts[ev] = 0

evt_counts = evt_counts[TOP_EVENTS].reset_index()
evt_counts = evt_counts.rename(columns={ev: f"event_{ev}_count" for ev in TOP_EVENTS})

base = base.merge(evt_counts, on="bucket_id", how="left")

count_cols = ["n_events", "n_failed", "n_machines", "n_collections"] + \
             [f"event_{ev}_count" for ev in TOP_EVENTS]

for c in count_cols:
    base[c] = base[c].fillna(0).astype("int64")

base = base.sort_values("bucket_id").reset_index(drop=True)

log(f"Base shape: {base.shape}")


# =========================
# 5) Série contínua
# =========================
bmin = int(base["bucket_id"].min())
bmax = int(base["bucket_id"].max())

full = pd.DataFrame({"bucket_id": np.arange(bmin, bmax + 1)})
full["bucket_start_us"] = full["bucket_id"] * BUCKET_US

series = full.merge(base, on=["bucket_id", "bucket_start_us"], how="left")

for c in count_cols:
    series[c] = series[c].fillna(0).astype("int64")

log(f"Série shape: {series.shape}")


# =========================
# 6) Persistência
# =========================
BASE_FILE = FEATURES_PATH / "window_5min_base.parquet"
SERIES_FILE = FEATURES_PATH / "window_5min_series.parquet"

base.to_parquet(BASE_FILE, compression="snappy", index=False)
series.to_parquet(SERIES_FILE, compression="snappy", index=False)

summary = {
    "rows_in": int(len(df)),
    "base_rows": int(len(base)),
    "series_rows": int(len(series)),
    "bucket_id_min": bmin,
    "bucket_id_max": bmax,
    "bucket_span": int(bmax - bmin + 1),
    "gap_buckets": int(len(series) - len(base)),
    "avg_events_per_bucket": float(base["n_events"].mean()),
    "avg_failed_per_bucket": float(base["n_failed"].mean()),
    "top_events": TOP_EVENTS
}

summary_file = REPORTS_PATH / "03_window_5min_base_summary.json"
summary_file.write_text(json.dumps(summary, indent=2, ensure_ascii=False))

log("Notebook 03 finalizado com sucesso.")
base.head()

[Bootstrap] Drive já montado.
[03_window_5min_base] Shape entrada: (349151, 23)
[03_window_5min_base] bucket range: 12..8929
[03_window_5min_base] Buckets distintos: 8917
[03_window_5min_base] Base shape: (8917, 16)
[03_window_5min_base] Série shape: (8918, 16)
[03_window_5min_base] Notebook 03 finalizado com sucesso.


,bucket_id,bucket_start_us,n_events,n_failed,n_machines,n_collections,mean_priority,mean_req_cpus,mean_req_mem,event_FAIL_count,event_SCHEDULE_count,event_FINISH_count,event_ENABLE_count,event_LOST_count,event_EVICT_count,event_KILL_count
0,12,3600000000,25,6,25,20,270.160000,0.019362,0.020901,6,2,2,4,11,0,0
1,13,3900000000,46,13,43,23,209.717391,0.007103,0.018461,13,0,13,7,12,0,1
2,14,4200000000,34,11,32,20,189.382353,0.014036,0.015593,11,1,10,7,5,0,0
3,15,4500000000,44,10,41,29,249.522727,0.008850,0.003220,10,0,9,10,15,0,0
4,16,4800000000,22,5,22,19,219.045455,0.011749,0.003593,5,1,6,4,5,0,1
